In [1]:
import gzip
import pickle
import os
from tqdm.notebook import tqdm
import requests as re
import pandas as pd
from ethereum import *
from web3 import Web3

In [2]:
path_dir = './data/'
os.makedirs(path_dir, exist_ok=True)

In [3]:
def load_dataset(contract_name):
    with gzip.open(path_dir + 'events_' + contract_name + '.pkl.gz', 'rb') as f:
        events = pickle.load(f)
    return events


def parse_common_attributes(event):
    data = dict()
    data['blockNumber'] = event['blockNumber']
    data['transactionHash'] = event['transactionHash'].hex().lower()
    data['blockHash'] = event['blockHash'].hex().lower()
    data['address'] = event['address'].lower()
    data['transactionIndex'] = event['transactionIndex']
    data['logIndex'] = event['logIndex']
    data['event'] = event['event']
    return data


def transfer_to_dataframe(events):
    # Convert Transfer events data to dataframe
    df = list()
    for event in tqdm(events, desc='Loading Transfer events'):
        data = parse_common_attributes(event)

        data['from'] = event['args']['from'].lower()
        data['to'] = event['args']['to'].lower()
        data['value'] = event['args']['value']
        df.append(data)
    return pd.DataFrame(df)


def approval_to_dataframe(events):
    # Convert Approval events data to dataframe
    df = list()
    for event in tqdm(events, desc='Loading Approval events'):
        data = parse_common_attributes(event)

        data['owner'] = event['args']['owner'].lower()
        data['spender'] = event['args']['spender'].lower()
        data['value'] = event['args']['value']
        df.append(data)
    return pd.DataFrame(df)

def swap_to_dataframe(events):
    # Convert Swap events data to dataframe
    df = list()
    for event in tqdm(events, desc='Loading Swap events'):
        data = parse_common_attributes(event)

        data['sender'] = event['args']['sender'].lower()
        data['recipient'] = event['args']['recipient'].lower()
        data['amount0'] = event['args']['amount0']
        data['amount1'] = event['args']['amount1']
        data['sqrtPriceX96'] = event['args']['sqrtPriceX96']
        data['liquidity'] = event['args']['liquidity']
        data['tick'] = event['args']['tick']
        df.append(data)
    return pd.DataFrame(df)


In [4]:
def load_blocks():
    dfs = list()
    filenames = [filename for filename in os.listdir(
        path_dir) if filename.startswith('blocks_') and filename.endswith('.csv.gz')]
    for filename in filenames:
        df = pd.read_csv(path_dir + filename, sep=';', compression='gzip')
        dfs.append(df)
    dfs = pd.concat(dfs)
    dfs.drop_duplicates(inplace=True)
    dfs['timestamp'] = pd.to_datetime(dfs['timestamp'])
    return dfs

# Loading account balances history for specific addresses
def compute_account_balances(transfer_df):
    balances_history_dict = dict()
    for _, row in tqdm(transfer_df.iterrows(), desc='Loading balances', total=transfer_df.shape[0]):
        if row['from'] not in balances_history_dict:
            balances_history_dict[row['from']] = {
                'current': 0, 'history': [], 'n_from': 0, 'n_to': 0}
        if row['to'] not in balances_history_dict:
            balances_history_dict[row['to']] = {
                'current': 0, 'history': [], 'n_from': 0, 'n_to': 0}

        balance = row['value']/1e18

        balances_history_dict[row['from']]['history'].append(
            {'block_number': row['blockNumber'], 'timestamp': row['timestamp'],
             'balance': balances_history_dict[row['from']]['current']-balance})
        balances_history_dict[row['to']]['history'].append(
            {'block_number': row['blockNumber'], 'timestamp': row['timestamp'],
             'balance': balances_history_dict[row['to']]['current']+balance})

        balances_history_dict[row['from']]['n_from'] += 1
        balances_history_dict[row['to']]['n_to'] += 1

        balances_history_dict[row['from']]['current'] -= balance
        balances_history_dict[row['to']]['current'] += balance

    print("There are in total {} addresses".format(len(balances_history_dict)))
    return balances_history_dict

In [5]:
contract_settings = {
    'Uniswap-DAI': {'address': to_checksum_address('0xc2e9f25be6257c210d7adf0d4cd6e3e881ba25f8'), 'start': 12_369_854, 'end': 18_362_881, 'decimals': 18},
    # 'UniswapV3Factory': {'address': to_checksum_address('0x1f98431c8ad98523631ae4a59f267346ea31f984'), 'start': 12_369_621, 'end': 18_362_881, 'decimals': 18},
    # 'Multicall2': {'address': to_checksum_address('0x5ba1e12693dc8f9c48aad8770482f4739beed696'), 'start': 12_336_033, 'end': 18_362_881, 'decimals': 18},
    # 'ProxyAdmin': {'address': to_checksum_address('0xb753548f6e010e7e680ba186f9ca1bdab2e90cf2'), 'start': 12_369_623, 'end': 18_362_881, 'decimals': 18},
    # 'TickLens': {'address': to_checksum_address('0xbfd8137f7d1516d3ea5ca83523914859ec47f573'), 'start': 12_369_625, 'end': 18_362_881, 'decimals': 18},
    # 'Quoter': {'address': to_checksum_address('0xb27308f9f90d607463bb33ea1bebb41c27ce5ab6'), 'start': 12_369_627, 'end': 18_362_881, 'decimals': 18},
    # 'SwapRouter': {'address': to_checksum_address('0xe592427a0aece92de3edee1f18e0157c05861564'), 'start': 12_369_634, 'end': 18_362_881, 'decimals': 18},
    # 'NFTDescriptor': {'address': to_checksum_address('0x42b24a95702b9986e82d421cc3568932790a48ec'), 'start': 12_369_642, 'end': 18_362_881, 'decimals': 18},
    # 'NonfungibleTokenPositionDescriptor': {'address': to_checksum_address('0x91ae842a5ffd8d12023116943e72a606179294f3'), 'start': 12_369_645, 'end': 18_362_881, 'decimals': 18},
    # 'TransparentUpgradeableProxy': {'address': to_checksum_address('0xee6a57ec80ea46401049e92587e52f5ec1c24785'), 'start': 12_369_648, 'end': 18_362_881, 'decimals': 18},
    # 'NonfungiblePositionManager': {'address': to_checksum_address('0xc36442b4a4522e871399cd717abdd847ab11fe88'), 'start': 12_369_651, 'end': 18_362_881, 'decimals': 18},
    # 'V3Migrator': {'address': to_checksum_address('0xa5644e29708357803b5a882d272c41cc0df92b34'), 'start': 12_369_654, 'end': 18_362_881, 'decimals': 18},
}


# Data gathering from archive node


In [6]:
eth_node = os.environ['ETHEREUM_NODE_URL']

adapter = re.adapters.HTTPAdapter(pool_connections=20, pool_maxsize=20)
session = requests.Session()
session.mount('http://', adapter)
session.mount('https://', adapter)

w3 = Web3(Web3.HTTPProvider(eth_node, session=session,
          request_kwargs={'timeout': 60}))

print("Is connected to Ethereum node: ", w3.is_connected())
print("The most recent block is: ", w3.eth.block_number)

Is connected to Ethereum node:  True
The most recent block is:  18392529


In [7]:
contracts = {}
for contract in contract_settings:
    contracts[contract] = get_contract(
        w3, contract_settings[contract]['address'])

In [8]:
for contract in contracts:
    print(contract)
    for event in contracts[contract].events:
        print('\t', event.event_name)

Uniswap-DAI
	 Burn
	 Collect
	 CollectProtocol
	 Flash
	 IncreaseObservationCardinalityNext
	 Initialize
	 Mint
	 SetFeeProtocol
	 Swap


In [ ]:
# Gather all contract events from contract_settings
for contract_name in contract_settings:
    print('====== ', contract_name, ' ======')
    events = get_all_events_from_contract(contracts[contract_name],
                                          start_block=contract_settings[contract_name]['start'],
                                          end_block=contract_settings[contract_name]['end'],
                                          batch_size=1000,
                                          max_workers=20,
                                          events=None)
    with gzip.open(path_dir + 'events_' + contract_name + '.pkl.gz', 'wb') as f:
        pickle.dump(events, f)


In [9]:
# Load all contract events from contract_settings
contract_events = dict()
for contract_name in tqdm(contract_settings):
    contract_events[contract_name] = load_dataset(contract_name)

  0%|          | 0/1 [00:00<?, ?it/s]

In [10]:
for contract_name in contract_events:
    for event_name in contract_events[contract_name]:
        print(
            f"There are {len(contract_events[contract_name][event_name])} {event_name} events")

There are 10071 Burn events
There are 8980 Collect events
There are 0 CollectProtocol events
There are 34 Flash events
There are 11 IncreaseObservationCardinalityNext events
There are 1 Initialize events
There are 9335 Mint events
There are 0 SetFeeProtocol events
There are 183423 Swap events


In [12]:
all_txs = set()
for contract_name in contract_events:
    for event_name in contract_events[contract_name]:
        txs = set()
        for event in contract_events[contract_name][event_name]:
            txs.add(event['transactionHash'].hex())
            all_txs.add(event['transactionHash'].hex())
        print("There are", len(txs), "unique transactions for", event_name)
    print("There are", len(all_txs), "unique transactions in total")


There are 8951 unique transactions for Burn
There are 8719 unique transactions for Collect
There are 0 unique transactions for CollectProtocol
There are 34 unique transactions for Flash
There are 11 unique transactions for IncreaseObservationCardinalityNext
There are 1 unique transactions for Initialize
There are 9179 unique transactions for Mint
There are 0 unique transactions for SetFeeProtocol
There are 183138 unique transactions for Swap
There are 200831 unique transactions in total


In [13]:
# block_min = float('inf')
# block_max = 0
# for contract_name in contract_settings:
#     block_min = min(block_min, contract_settings[contract_name]['start'])
#     block_max = max(block_max, contract_settings[contract_name]['end'])
# print("The min block and max block are: ", block_min, block_max)

In [14]:
# # Get blocks
# blocks = get_blocks(w3, range(block_min, block_max))
# blocks

In [15]:
txs = list()
for contract_name in tqdm(contract_settings):
    events = load_dataset(contract_name)
    for event in events:
        txs += list(map(lambda tx: tx['transactionHash'].hex().lower(), events[event]))
print("There are {} transactions".format(len(txs)))
txs = list(set(txs))
print("There are {} unique transactions".format(len(txs)))

  0%|          | 0/1 [00:00<?, ?it/s]

There are 211855 transactions
There are 200831 unique transactions


In [17]:
# Code to gather all transactions that triggered the events
c = 1
for index, thumb in tqdm(get_batch_intervals(0, len(txs), 200_000)):
    txs_data = get_transactions(w3, txs[index:thumb])
    with gzip.open(path_dir + 'txs_' + str(c) + '.pkl.gz', 'wb') as f:
        pickle.dump(txs_data, f)
    c += 1


  0%|          | 0/2 [00:00<?, ?it/s]

Gathering transactions...:   0%|          | 0/199999 [00:00<?, ?it/s]

In [17]:
contract_names = sorted([filename.split('_')[1][:-7] for filename in os.listdir(
    path_dir) if filename.startswith('events_')])
contract_names

['Uniswap-DAI']

In [18]:
# An example of how to parse the data to a pandas dataframe
swap_df = swap_to_dataframe(contract_events['Uniswap-DAI']['Swap'])
swap_df.head()

Loading Swap events:   0%|          | 0/183423 [00:00<?, ?it/s]

,blockNumber,transactionHash,blockHash,address,transactionIndex,logIndex,event,sender,recipient,amount0,amount1,sqrtPriceX96,liquidity,tick
0,12369879,0x2bdb4298b35adf058a38dfbe85470f67da1cb76e1694...,0x7d81e60e5a2296dc38f36e343a7f3e416b1fc2f76656...,0xc2e9f25be6257c210d7adf0d4cd6e3e881ba25f8,40,91,Swap,0xe592427a0aece92de3edee1f18e0157c05861564,0x3b8ccaa89fcd432f1334d35b10ff8547001ce3e5,-33854155678824490173,10000000000000000,1364573512386034424627810688,80059851033970806503,-81234
1,12370126,0x1e268cf8d859b38f50d338aeaad6973b1335f5999f67...,0x611e414d1b1238fdef1bea14555e42e72cb858e6957b...,0xc2e9f25be6257c210d7adf0d4cd6e3e881ba25f8,66,92,Swap,0xe592427a0aece92de3edee1f18e0157c05861564,0x74aa01d162e6dc6a657cac857418c403d48e2d77,-336093753482074,100000000000,1364573546772589608591144427,229713263233648638236,-81234
2,12371051,0x5cdf9ec984fb0fe801cc529c003bcdd5271857103c19...,0x3a83712fa13bed0da97d632a3a668860e904ad00c371...,0xc2e9f25be6257c210d7adf0d4cd6e3e881ba25f8,111,361,Swap,0xe592427a0aece92de3edee1f18e0157c05861564,0x0459b3fbf7c1840ee03a63ca4aa95de48322322e,-331043258762748247602,100000000000000000,1385391852998664492209230571,379427976363351507096,-80932
3,12371223,0x3d71709ade6b409b2223c6a3ef7356646f615d894802...,0x40ec7fffbae4b145d12f8c8a8e340b3d3bfd81ffbd77...,0xc2e9f25be6257c210d7adf0d4cd6e3e881ba25f8,185,308,Swap,0xe592427a0aece92de3edee1f18e0157c05861564,0x3b8ccaa89fcd432f1334d35b10ff8547001ce3e5,-10026667527943594868,3076438757863740,1386032315440134219844025713,379427976363351507096,-80922
4,12375729,0xb349853fe3e9c6069d8264202c9d68ad5b737919f0ca...,0x27b9278a88afa5ddcccf3c9878d3368d1ce4c058d477...,0xc2e9f25be6257c210d7adf0d4cd6e3e881ba25f8,53,151,Swap,0xe592427a0aece92de3edee1f18e0157c05861564,0x22f9dcf4647084d6c31b2765f6910cd85c178c18,10682019392695845455577,-3254228708039319328,1383838353700367458448734141,117516434432686612962365,-80954
